In [ ]:
### Importar librerias

In [11]:
import pandas as pd
import numpy as np
from bdatos import Bdatos
from ingestar import Ingestar

import warnings
warnings.filterwarnings("ignore")

In [14]:
### Descarga y carga del dataset desde Kaggle

# Crear instancia del ingestar si no existe
try:
    ingestar
    print("Instancia de Ingestar ya existe")
except:
    ingestar = Ingestar()
    print("Nueva instancia de Ingestar creada")

# Descargar el dataset desde Kaggle
try:
    print("Descargando dataset desde Kaggle...")
    dataset_path = ingestar.download_dataset_zip("samuelotiattakorah/agriculture-crop-yield")
    print(f"Dataset descargado en: {dataset_path}")
    
    # Extraer archivos si es necesario
    data_dir = ingestar.extract_zip_files(dataset_path)
    print(f"Archivos extraídos en: {data_dir}")
    
    # Cargar como DataFrame
    df_agriculture = ingestar.load_dataset_as_dataframe(data_dir)
    print(f"Dataset cargado como DataFrame: {df_agriculture.shape}")
    print("Columnas disponibles:", list(df_agriculture.columns))
    
except Exception as e:
    print(f"Error al descargar/cargar el dataset: {e}")
    print("Verificar conexión a internet y credenciales de Kaggle")

Instancia de Ingestar ya existe
Descargando dataset desde Kaggle...
Descargando dataset desde Kaggle...
Ruta al dataset: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Dataset descargado en: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Archivos encontrados en la descarga: ['crop_yield.csv']
Se detectaron archivos CSV directamente en la carpeta descargada.
Archivos extraídos en: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Leyendo CSV /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1/crop_yield.csv ...
Ruta al dataset: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Dataset descargado en: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Archivos encontrados en la descarga: ['crop_yield.csv']
Se detectaron 

In [18]:
### Verificar estado actual del ingestar y base de datos

print("VERIFICANDO ESTADO ACTUAL DEL SISTEMA")
print("=" * 50)

# Verificar si ya tenemos instancias creadas
try:
    print(f"Ingestar: {type(ingestar)}")
    print("Instancia de Ingestar disponible")
except:
    print("No hay instancia de Ingestar disponible")
    ingestar = Ingestar()
    print("Nueva instancia de Ingestar creada")

# Verificar variables del dataset
print("\nINFORMACIÓN DEL DATASET:")
try:
    print(f"Dataset path: {dataset_path}")
    print(f"Data directory: {data_dir}")
    print(f"Forma del DataFrame: {df_agriculture.shape}")
    print("Dataset cargado correctamente")
    
    # Mostrar información básica del dataset
    print("\nINFORMACIÓN BÁSICA DEL DATASET:")
    print(f"Columnas: {list(df_agriculture.columns)}")
    print(f"Primeras 3 filas:")
    print(df_agriculture.head(3))
    
except Exception as e:
    print(f"Error con el dataset: {e}")

# Verificar estado de la base de datos
print("\nESTADO DE LA BASE DE DATOS:")
try:
    from bdatos import Bdatos
    
    # Si ya existe una instancia de BD
    if 'bd_cultivos' in locals() or 'bd_cultivos' in globals():
        print("Instancia de base de datos encontrada")
        tablas = bd_cultivos.listar_tablas()
        print(f"Tablas existentes: {tablas}")
        
        for tabla in tablas:
            conteo = bd_cultivos.contar_filas(tabla)
            print(f"  - {tabla}: {conteo} filas")
    else:
        print("No se encontró instancia de base de datos")
        
except Exception as e:
    print(f"Error al verificar base de datos: {e}")

print("\n" + "=" * 50)

VERIFICANDO ESTADO ACTUAL DEL SISTEMA
Ingestar: <class 'ingestar.Ingestar'>
Instancia de Ingestar disponible

INFORMACIÓN DEL DATASET:
Dataset path: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Data directory: /home/codespace/.cache/kagglehub/datasets/samuelotiattakorah/agriculture-crop-yield/versions/1
Forma del DataFrame: (1000000, 10)
Dataset cargado correctamente

INFORMACIÓN BÁSICA DEL DATASET:
Columnas: ['Region', 'Soil_Type', 'Crop', 'Rainfall_mm', 'Temperature_Celsius', 'Fertilizer_Used', 'Irrigation_Used', 'Weather_Condition', 'Days_to_Harvest', 'Yield_tons_per_hectare']
Primeras 3 filas:
  Region Soil_Type    Crop  Rainfall_mm  Temperature_Celsius  Fertilizer_Used  \
0   West     Sandy  Cotton   897.077239            27.676966            False   
1  South      Clay    Rice   992.673282            18.026142             True   
2  North      Loam  Barley   147.998025            29.794042            False   

   Irrigation_Used W

In [3]:
### Creación de la base de datos, las tablas e inserción de datos

In [16]:
# Importar la clase Bdatos corregida
try:
    import importlib
    import bdatos
    importlib.reload(bdatos)
    from bdatos import Bdatos
    
    print("Clase Bdatos importada correctamente")
    
    # Crear instancia de la base de datos
    bd_test = Bdatos("static/db/proyecto.db")
    
    # Probar inserción de una muestra del dataset
    df_muestra = df_agriculture.head(1000)  # Tomar solo 1000 filas para prueba
    
    print(f"Insertando muestra de {len(df_muestra)} filas en la base de datos...")
    exito = bd_test.insertar_dataframe(df_muestra, "Cultivos")
    
    if exito:
        # Verificar que se insertó correctamente
        print("Verificando inserción:")
        tablas = bd_test.listar_tablas()
        print(f"Tablas creadas: {tablas}")
        
        for tabla in tablas:
            filas = bd_test.contar_filas(tabla)
            print(f"  - {tabla}: {filas} filas")
        
        # Hacer una consulta de prueba
        print("\nConsulta de prueba - primeras 5 filas:")
        resultado = bd_test.consultar("SELECT * FROM Cultivos LIMIT 5")
        print(resultado)
        
        # Consulta estadística
        print("\nEstadísticas por región:")
        stats = bd_test.consultar("""
            SELECT Region, 
                   COUNT(*) as total_registros,
                   AVG(Yield_tons_per_hectare) as rendimiento_promedio,
                   AVG(Rainfall_mm) as lluvia_promedio
            FROM Cultivos 
            GROUP BY Region
        """)
        print(stats)
    
    bd_test.cerrar_conexion()
    
except Exception as e:
    print(f"Error al probar Bdatos: {e}")
    import traceback
    traceback.print_exc()

Clase Bdatos importada correctamente
 Conectado a la base de datos: static/db/proyecto.db
Insertando muestra de 1000 filas en la base de datos...
 DataFrame insertado en tabla 'Cultivos' (1000 filas)
Verificando inserción:
Tablas creadas: ['Cultivos']
  - Cultivos: 1000 filas

Consulta de prueba - primeras 5 filas:
 Consulta ejecutada exitosamente (5 filas)
  Region Soil_Type     Crop  Rainfall_mm  Temperature_Celsius  \
0   West     Sandy   Cotton   897.077239            27.676966   
1  South      Clay     Rice   992.673282            18.026142   
2  North      Loam   Barley   147.998025            29.794042   
3  North     Sandy  Soybean   986.866331            16.644190   
4  South      Silt    Wheat   730.379174            31.620687   

   Fertilizer_Used  Irrigation_Used Weather_Condition  Days_to_Harvest  \
0                0                1            Cloudy              122   
1                1                1             Rainy              140   
2                0         

In [ ]:
### Generar csv con la información de la base de datos

In [17]:
# Exportar datos de la base de datos a CSV
# Flujo: Dataset -> SQLite -> CSV

try:
    # Conectar a la base de datos existente
    bd_export = Bdatos("static/db/proyecto.db")
    
    # Consultar todos los datos de la tabla Cultivos
    print("Consultando datos de la tabla Cultivos...")
    df_cultivos = bd_export.consultar("SELECT * FROM Cultivos")
    
    if df_cultivos is not None and len(df_cultivos) > 0:
        # Definir la ruta de exportación
        export_path = "static/db/export.csv"
        
        # Exportar a CSV
        df_cultivos.to_csv(export_path, index=False)
        
        print(f"Datos exportados exitosamente a: {export_path}")
        print(f"Total de filas exportadas: {len(df_cultivos)}")
        print(f"Total de columnas: {len(df_cultivos.columns)}")
        
        # Verificar que el archivo se creó correctamente
        import os
        if os.path.exists(export_path):
            file_size = os.path.getsize(export_path) / 1024  # Tamaño en KB
            print(f"Archivo creado exitosamente - Tamaño: {file_size:.2f} KB")
            
            # Mostrar las primeras líneas del CSV para verificación
            print(f"\nPrimeras 5 líneas del archivo CSV generado:")
            with open(export_path, 'r') as f:
                for i, line in enumerate(f):
                    if i < 5:
                        print(f"Línea {i+1}: {line.strip()}")
                    else:
                        break
        else:
            print("Error: No se pudo crear el archivo CSV")
    
    else:
        print("No se encontraron datos en la tabla Cultivos")
    
    # Cerrar conexión
    bd_export.cerrar_conexion()
    
except Exception as e:
    print(f"Error al exportar a CSV: {e}")
    import traceback
    traceback.print_exc()

print("\nFlujo completado: Dataset -> SQLite -> CSV")

 Conectado a la base de datos: static/db/proyecto.db
Consultando datos de la tabla Cultivos...
 Consulta ejecutada exitosamente (1000 filas)
Datos exportados exitosamente a: static/db/export.csv
Total de filas exportadas: 1000
Total de columnas: 10
Archivo creado exitosamente - Tamaño: 84.18 KB

Primeras 5 líneas del archivo CSV generado:
Línea 1: Region,Soil_Type,Crop,Rainfall_mm,Temperature_Celsius,Fertilizer_Used,Irrigation_Used,Weather_Condition,Days_to_Harvest,Yield_tons_per_hectare
Línea 2: West,Sandy,Cotton,897.0772391101236,27.676966373377603,0,1,Cloudy,122,6.555816258223593
Línea 3: South,Clay,Rice,992.6732816189208,18.02614225436302,1,1,Rainy,140,8.5273409063236
Línea 4: North,Loam,Barley,147.9980252926104,29.79404241557257,0,0,Sunny,106,1.127443335982929
Línea 5: North,Sandy,Soybean,986.8663313367324,16.64419019137728,0,1,Rainy,146,6.517572507555278
Conexión cerrada

Flujo completado: Dataset -> SQLite -> CSV


In [ ]:
#Prueba general de información

In [23]:
### Análisis comparativo: Mejores cultivos por región con restricción de 120 días de cosecha

try:
    bd_final = Bdatos("static/db/proyecto.db")
    
    # Consulta para encontrar el mejor cultivo de cada región
    mejor_por_region = """
    SELECT 
        Region,
        Crop as Mejor_Cultivo,
        ROUND(MAX(Yield_tons_per_hectare), 2) as Max_Rendimiento,
        Days_to_Harvest as Dias_Cosecha
    FROM Cultivos 
    WHERE Days_to_Harvest <= 120
    GROUP BY Region
    HAVING Yield_tons_per_hectare = MAX(Yield_tons_per_hectare)
    ORDER BY Max_Rendimiento DESC
    """
    
    print("🏆 CAMPEONES POR REGIÓN (Máximo 120 días)")
    print("=" * 50)
    
    campeones = bd_final.consultar(mejor_por_region)
    
    if campeones is not None:
        print("Ranking de regiones por mejor rendimiento individual:")
        print(f"{'Pos':<3} {'Región':<6} {'Cultivo':<8} {'Tons/Ha':<8} {'Días'}")
        print("-" * 40)
        
        for i, (_, row) in enumerate(campeones.iterrows(), 1):
            print(f"{i:<3} {row['Region']:<6} {row['Mejor_Cultivo']:<8} {row['Max_Rendimiento']:<8} {row['Dias_Cosecha']}")
    
    # Resumen estadístico global
    resumen_global = """
    SELECT 
        COUNT(DISTINCT Crop) as Total_Cultivos,
        ROUND(AVG(Yield_tons_per_hectare), 2) as Rendimiento_Promedio_Global,
        ROUND(AVG(Days_to_Harvest), 1) as Dias_Promedio_Global,
        COUNT(*) as Total_Registros_Filtrados
    FROM Cultivos 
    WHERE Days_to_Harvest <= 120
    """
    
    stats_global = bd_final.consultar(resumen_global)
    
    if stats_global is not None:
        print(f"\n📊 ESTADÍSTICAS GLOBALES (≤120 días):")
        print("-" * 35)
        row = stats_global.iloc[0]
        print(f"Cultivos diferentes: {row['Total_Cultivos']}")
        print(f"Rendimiento promedio: {row['Rendimiento_Promedio_Global']} tons/ha")
        print(f"Días promedio cosecha: {row['Dias_Promedio_Global']} días")
        print(f"Total registros analizados: {row['Total_Registros_Filtrados']}")
    
    bd_final.cerrar_conexion()
    
    print(f"\n✅ ANÁLISIS COMPLETADO")
    print("Consulta generacional exitosa: Top 3 cultivos por región con restricción de 120 días máximo")
    
except Exception as e:
    print(f"Error en análisis final: {e}")

 Conectado a la base de datos: static/db/proyecto.db
🏆 CAMPEONES POR REGIÓN (Máximo 120 días)
 Consulta ejecutada exitosamente (4 filas)
Ranking de regiones por mejor rendimiento individual:
Pos Región Cultivo  Tons/Ha  Días
----------------------------------------
1   East   Maize    9.14     64
2   West   Barley   8.5      65
3   South  Rice     8.14     71
4   North  Wheat    8.13     66
 Consulta ejecutada exitosamente (1 filas)

📊 ESTADÍSTICAS GLOBALES (≤120 días):
-----------------------------------
Cultivos diferentes: 6.0
Rendimiento promedio: 4.64 tons/ha
Días promedio cosecha: 88.8 días
Total registros analizados: 667.0
Conexión cerrada

✅ ANÁLISIS COMPLETADO
Consulta generacional exitosa: Top 3 cultivos por región con restricción de 120 días máximo
